In [9]:
#By Hasan Hashim
# Importting the following follows
import paramiko, os, re
from datetime import datetime
from getpass import getpass


# Create the password prompt
thePass = getpass(prompt="Please enter your SSH password: ")

# Host information
host = "192.168.6.71"
port = 2222
username = "hasan.hashim"
password = thePass
try:

    ssh = paramiko.SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy()) 
    ssh.connect(host, port, username, password)

except paramiko.AuthenticationException:
    print("Authentication Failed.")


In [3]:
sftp = ssh.open_sftp()

# Check if the file already exists
try:
    # Hwew we are checking if the file is already exists or not
    sftp.stat("SYS-320\Week09\class\kraken")
    # Helpful message for the user
    print("- file already exists.")
except IOError:
    # if the file does not exists then will do the following
    sftp.put("./kraken", "/home/hasan.hashim/kraken")
    print("- The file was upload successful")





# Here I'm writting a fucntion to save the out put inside a file
# The point of this function so to make easy for me nto save the out put
# So I don't have to write f.write every time I want to save data inside a file.
def write(s, output):
    print(f"- Writing the output to logfile called {s}_output.txt")
    # Here we trying to name the file and let the user write the first half of the name.
    with open(f"{s}_output.txt", "a") as f:
        f.write(f"\n### {s} output - ")
        # Writting the time and the date
        f.write(datetime.now().strftime("%Y-%m-%d %H:%M:%S") + " ###\n")
        # adding the out put
        f.write(output)

[-] Uploading kraken to the remote host...
[-] Upload successful


In [4]:
# This is for testting only. I just wanted to make suer that the file was uploaded
command = ['ls','pwd']
for eachCMD in command:
    # Get the output from the command
    stdin, stdout, stderr = ssh.exec_command(eachCMD)

    # Get results from stdout
    lines = stdout.readlines()
    #print(lines)

    # Convert the lsiti to a string
    output = ''.join(lines)

    # Header
    sepHeader = '' + '### BEGIN ' + eachCMD + ' ###\n\n'

    # Footer
    sepFooter = '' + '### END ' + eachCMD + ' ###\n\n'

    # Concatenate the header, output, and footer
    cmd_output = sepHeader + output + sepFooter

    # Save the cmd_output to a file
    # with open('ssh_results.txt', 'a') as f:
    #     f.write(cmd_output)
    print(cmd_output)

### BEGIN ls ###

kraken
### END ls ###


### BEGIN pwd ###

/home/hasan.hashim
### END pwd ###




In [13]:

ssh.exec_command("chmod +x /home/hasan.hashim/kraken")
# Here are the command to use when running kraken
kraken_command = "sudo -S /home/hasan.hashim/kraken --folder /usr/bin --folder  --folder /usr/sbin/   --folder /usr/local/bin  --folder /sbin  --folder /usr/local/sbin  --folder /bin"
# Here I'm tryying to run and get the output from kraken
print("[-] Running kraken on the remote host...")
stdin, stdout, stderr = ssh.exec_command(kraken_command)
# We have to run the commands as a root so I used the following line to do that
stdin.write(password + "\n")
# Getting the information from the stdout and stored in lines
lines = stdout.readlines()
print("- The execution was completed")
# Here I'm trying to make it look orgnized
ansi_escape = re.compile(r'\x1B(?:[@-Z\\-_]|\[[0-?]*[ -/]*[@-~])')
output = ansi_escape.sub("", "".join(lines))
# Here I'm using the function write to save the output inside a file
write("kraken", output)



[-] Running kraken on the remote host...
[-] Execution of kraken completed
[-] Writing kraken output to logfile kraken_output.txt


In [15]:
# Get only pids for detected processes to pipe into lsof

# Trying to figuer out if there are any processes running containing matches from the task above for ThugStyle activity, use lsof to list all open files for that process.  Look for detections that include: pid=XXXX
pids = re.findall(r"pid=(\d+)(?!.*snapd)", output)
# Here I'm printting the detected process that my program saw
print(f"- Detected {len(pids)} process(es) with pid(s) {list(map(int, pids))}")
# Here I'm printting the files that was opened
print(f"- Getting list of open files for {len(pids)} process(es)...")
# Here I'm tryying to send an command in to the host to run the pids that I got
lsof_command = f"sudo -S lsof -p {','.join(pids)}"
# Here I'm getting the output from lsof command
stdin, stdout, stderr = ssh.exec_command(lsof_command)
# Becouse we unning command with sudo, the user password
stdin.write(password + "\n")
lsof_output = "".join(stdout.readlines())

# Here I'm using the function write to save the output inside a file
write("lsof", lsof_output)

[-] Detected 1 process(es) with pid(s) [561559]
[-] Getting list of open files for 1 process(es)...
[-] Writing kraken output to logfile lsof_output.txt


In [16]:
# Here I'm processing lsof output to check for suspicious usernames
print("- looking for suspicious usernames in lsof outputs")

# Here I create a format for ThugStyle username
thusername = re.compile(r"^[A-Za-z]\.[A-Za-z]+")
# Getting a list
sus_usernames = []

# Now we need to create a output directory to be orgnized 
if not os.path.exists("./output"):
    os.mkdir("output")

# looking through a lsof outputs to check for usernames
for line in lsof_output.splitlines():
    # suspicious username pattern
    if re.search(thusername, line.split()[2]):
        fields = line.split()











        
        # Check for presence of username in the list, to avoid duplicate output
        if fields[2] not in sus_usernames:
            print(f". Suspicious username found in lsof output: {fields[2]}")
            sus_usernames.append(fields[2])
        # Check lsof entry for download requirements
        if fields[3] != "DEL" and fields[4] == "REG":
            out_filename = "output/" + fields[8].split('/')[-1]
            # Check that file hasn't already been downloaded
            if not os.path.exists(out_filename):
                print(f"[!] Downloading suspicious file: {fields[8]}")
                sftp.get(fields[8], out_filename)
            else:
                print(f"[-] Skipping file {fields[8]}, already downloaded")

print("[-] Download(s) complete")

[-] Checking for suspicious usernames in lsof output
[!] Suspicious username found in lsof output: p.dalton
[!] Downloading suspicious file: /usr/local/bin/egrep
[!] Downloading suspicious file: /usr/lib/x86_64-linux-gnu/libc-2.31.so
[!] Downloading suspicious file: /usr/lib/x86_64-linux-gnu/libpthread-2.31.so
[!] Downloading suspicious file: /usr/lib/x86_64-linux-gnu/ld-2.31.so
[!] Downloading suspicious file: /home/p.dalton/.d.log
[-] Skipping file /home/p.dalton/.d.log, already downloaded
[!] Downloading suspicious file: /home/p.dalton/d.pid
[-] Download(s) complete


In [17]:
# Check other locations on the system for ThugStyle activity
search_commands = ["ps aux", "cat /etc/passwd", "sudo -S cat /var/log/auth.log"]

# Redefine thugstyle regex to work with multi-field lines
thusername = re.compile(r"(^|[^A-Za-z])([A-Za-z]\.[A-Za-z]+)")

for command in search_commands:
    # Print command being run
    print(f"[-] Running command: {command}")
    
    # Run and get the output for each command
    stdin, stdout, stderr = ssh.exec_command(command)
    
    # Enter password if command uses sudo
    if "sudo" in command:
        stdin.write(password + "\n")
    
    command_output = "".join(stdout.readlines())

    # Trim output to only results related to ThugStyle users
    thugstyle = re.findall(thusername, output)

    for thug in thugstyle:
        if thug not in sus_usernames:
            print(f"[!] Suspicious username found: {thug}")
            sus_usernames.append(thug)

    # Output full suspicious line
    for line in command_output.splitlines():
        if re.search(thusername, line):
            print(line)

[-] Running command: ps aux
p.dalton  560582  0.0  0.1  18364 18352 ?        Ss   Oct25   0:00 /lib/systemd/systemd --user
p.dalton  560590  0.0  1.0 182120 173192 ?       S    Oct25   0:00 (sd-pam)
p.dalton  561559  0.0  0.0 1003532 1212 ?        Ssl  Oct25   0:00 [go-daemon sample]
[-] Running command: cat /etc/passwd
p.dalton:x:1001:1001::/home/p.dalton:/bin/bash
[-] Running command: sudo -S cat /var/log/auth.log
Oct 30 17:13:25 sys320 sudo: alexander.brown : TTY=unknown ; PWD=/home/alexander.brown ; USER=root ; COMMAND=/usr/bin/ps -u p.dalton
Oct 30 20:16:15 sys320 sudo: alexander.brown : TTY=unknown ; PWD=/home/alexander.brown ; USER=root ; COMMAND=/usr/bin/ps -u p.dalton
Oct 30 20:16:16 sys320 sudo: alexander.brown : TTY=unknown ; PWD=/home/alexander.brown ; USER=root ; COMMAND=/usr/bin/lastlog -b 3 -u p.dalton
Oct 30 20:16:16 sys320 sudo: alexander.brown : TTY=unknown ; PWD=/home/alexander.brown ; USER=root ; COMMAND=/usr/bin/ls -d /home/p.dalton -al
Oct 30 20:18:33 sys320 sudo:

In [18]:
for username in sus_usernames:
    print(f"[-] Listing contents of home directory for {username}")
    stdin, stdout, stderr = ssh.exec_command(f"/usr/bin/ls -la /home/{username}")
    print("".join(stdout.readlines()))

#ssh.close()

[-] Listing contents of home directory for p.dalton
total 528
drwxr-xr-x  4 p.dalton p.dalton   4096 Oct 25 23:31 .
drwxr-xr-x 60 root     root       4096 Oct 31 10:12 ..
-rwxr-xr-x  1 p.dalton p.dalton    360 Oct 25 23:32 .bash_history
drwxr-xr-x  2 p.dalton p.dalton   4096 Feb 26  2022 .cache
-rwxr-xr-x  1 p.dalton p.dalton    850 Oct 25 23:32 .d.log
-rwxr-xr-x  1 p.dalton p.dalton 511476 Feb 18  2022 aqwerpuqwerj
-rw-r--r--  1 p.dalton p.dalton      6 Oct 25 23:32 d.pid
drwxr-xr-x  2 p.dalton p.dalton   4096 Oct 29 19:51 test-lk



In [19]:
ssh.close()